In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests

In [2]:
#Extract the all the new url inside html_link 
def extract_p_tags_45(html_link):
    page_link = urllib.request.urlopen(html_link).read()
    soup_link = BeautifulSoup(page_link)
    paragraphes = soup_link.findAll('a',href = re.compile('\d+'))
    list_tp = []

    for pa in paragraphes:
        # doesn't use the link of the main page (all issues)
        if(pa['href'] != '1200000e.html'):
            html_link = 'https://enb.iisd.org/vol12/'+pa['href']
            list_tp += extract_p_tags_rest(html_link)

    return list_tp


In [3]:
def extract_p_tags_66(html_link):
    page = urllib.request.urlopen(html_link).read()

    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page))
    #expression regulière pour enelevr dans les tags 
    page_string = re.sub('<!-- www design jeff anderson janderson@iisd.ca ---!>','',str(page_string).lower())
 
    soup = BeautifulSoup(page_string,'html.parser')
    list_tp = soup.find_all('p')
    #s =b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN""http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n'
    return list_tp

In [4]:
#Extract the <p> tag from a specific html link
def extract_p_tags_rest(html_link):
    page = urllib.request.urlopen(html_link).read()
    soup = BeautifulSoup(page,'html.parser')
    list_tp = soup.find_all('p')
    return list_tp

In [5]:
def extract_from_csv_p_tags(csv_file):
    f = open(csv_file)
    csv_f = csv.reader(f)
    k=0
    list_pt = []
    for row in csv_f:
        """
        #Extract for 0 < Issue# < 45 
        if(k>0 and int(row[4]) < 45 ):

            list_pt += extract_p_tags_45(row[6])

        
        if(k>0 and  45 < int(row[4]) and  int(row[4]) <=66 ):

            #check if URL still valid
            print(row[6])
            request = requests.get(row[6])
            if(request.status_code == 200):
                l = extract_p_tags_66(row[6])
                list_pt += l

        """
        #Extract for 66 < Issue# < 775
        if(k>0 and 66 < int(row[4]) and int(row[4]) <80):

            request = requests.get(row[6])
            if(request.status_code == 200):
                list_pt += extract_p_tags_rest(row[6])
        

        k +=1

    return list_pt

In [6]:
def clean_tp(sentence):
    s = re.sub("\r\n\s\s+"," ",sentence)
    s = re.sub("\r\n","",s)
    s = re.sub("\s\s+"," ",s)
    s = re.sub("\r\n","",s)
    

    return s

In [7]:
list_meetings = extract_from_csv_p_tags('list_meetings.csv')
list_p_tags = [clean_tp(str(line)) for line in list_meetings ]


In [9]:
# generate list_p_tags.txt file
outF = open("list_p_tags.txt", "w")
for line in list_p_tags:
    # write line to output file
    outF.write(str(line))
    outF.write("\n")
outF.close()

In [ ]:
s = re.sub("\s\s+"," ",s)
    s = re.sub("\r\n","",s)